# Interatomic force field (IFF) optimization for W-based metal alloys

Author: Lukas Vlcek

Start Date: 2018-04-22

In [1]:
from datetime import datetime ; print('Last update:', datetime.now())

Last update: 2018-04-23 00:58:33.810376


## 1. Target data overview

### Bulk properties of tungsten
from _Marinica et.al_ except for B (bulk modulus) and Pc (Cauchy pressure)

T=0K?

| Property | BCC | FCC |
| :---     | :---: | :---: |
| a_0 (A) | 3.1648 | 4.054 |
| E_coh (eV/atom) | -8.9 | -8.43 |
| C11 (GPa) | 523 | - |
| C12 (GPa) | 203 | - |
| C44 (GPa) | 160 | - |
| B (GPa) | 310.4 | - |
| Pc (GPa) | 21.9 | - |

### Defect properties
(from _Marinica et.al_)

Based on 128+-1 atoms

Defect | Energy (eV)
:--- | :---:
(111) | 10.53
(110) | 10.82
(100) | 12.87
OCT | 13.11
TET | 12.27
Vacancy | 3.49

### Atomic configurations and DFT data

1. Data from Marinica
  * 20 configurations generated from liquid Fe trajectory, N=113
  * DFT Forces
  * DFT Energy
2. Data from German (local minimum energy configs.)
  * Energy, Hessians?
  * Pure phases
    * BCC (54 atoms)
    * FCC (32 atoms)
  * Vacancy (53 atoms)
  * Interstitial defects 
    * 110, 111 (55 atoms)
    * 111_easy_core (135 atoms)?

## 2. Processing of raw data into HDF5 storage

In [2]:
import os
import glob
import re
import numpy as np
import h5py

# source of raw data
target_raw = '../data/target_raw'
german_raw = os.path.join(target_raw, 'dft_w')
marini_raw = os.path.join(target_raw, 'marinica_cfg')

# destination for processed data (hdf5 format)
target_proc = '../data/target_processed'
german_proc = os.path.join(target_proc, 'german')
marini_proc = os.path.join(target_proc, 'marini')

# read target data from Marinica, extract xyz, forces, energies
for file in glob.glob(os.path.join(marini_raw, '*.ref')):
    print(file)
    with open(file, 'r') as fi:
        ene = float(re.findall('\S+', fi.readline())[1])
        print('ene:', ene)
        fi.readline()
        #hrp.append([float(re.findall('\S+', fi.readline())[2]) for _ in range(lpmax)])
        for line in iter(fi.readline, ''):
            pass
        

../data/target_raw/marinica_cfg/cas10.ref
ene: -748.925969
../data/target_raw/marinica_cfg/cas11.ref
ene: -792.8714965
../data/target_raw/marinica_cfg/cas12.ref
ene: -880.6363416
../data/target_raw/marinica_cfg/cas13.ref
ene: -815.6129699
../data/target_raw/marinica_cfg/cas14.ref
ene: -749.7703982
../data/target_raw/marinica_cfg/cas15.ref
ene: -757.3972938
../data/target_raw/marinica_cfg/cas16.ref
ene: -759.2465558
../data/target_raw/marinica_cfg/cas17.ref
ene: -827.2314602
../data/target_raw/marinica_cfg/cas18.ref
ene: -753.0714903
../data/target_raw/marinica_cfg/cas19.ref
ene: -755.1659965
../data/target_raw/marinica_cfg/cas20.ref
ene: -770.3314257
../data/target_raw/marinica_cfg/cas4.ref
ene: -892.8326097
../data/target_raw/marinica_cfg/cas5.ref
ene: -754.7616929
../data/target_raw/marinica_cfg/cas6.ref
ene: -759.3048204
../data/target_raw/marinica_cfg/cas7.ref
ene: -758.0452513
../data/target_raw/marinica_cfg/cas8.ref
ene: -767.8930345
../data/target_raw/marinica_cfg/cas9.ref
ene: 

## 3. IFF functional form

### 3.1 Finnis-Sinclair potential

Energy of an N-particle configuration

$$ E = \sum_{i=1}^N \left[ \sum_{j>i}^N V\left(r_{ij}\right) + F\left(\rho_i\right) \right] $$

Pair interaction

$$ V\left(r_{ij}\right) = \left(r_{ij}-c\right)^2\left(c_0 + c_1r_{ij} + c_2r_{ij}^2\right),\,\, r_{ij}\le c $$


Embedded function

$$ F = -A\sqrt{\rho_i} $$

Effective electron density

$$ \rho_i = \sum_{j\ne i}^N \phi\left(r_{ij}\right) $$

Cohesive potential

$$ \phi\left(r_{ij}\right) = \left(r_{ij}-d\right)^2,\,\, r_{ij}\le d $$

** Starting W parameters: ** (Finnis and Sinclair, _Phil. Mag. A_, **50(1)**, 45-66, 1984)
* Fixed
  * d = 4.400224
  * c = 3.25
* To be optimized
  * A = 1.896373
  * c0 = 47.1346499
  * c1 = -33.7665655
  * c2 = 6.2541999

## 4. Loss function

1. Ab initio data (energies)

  $$  $$

2. Structural fluctuations
  * Fit Gaussian distributions described by bulk parameters
    * Mean values: lattice parameters (at a particular T
    * Variance: elastic constants

## 5. Optimization

* A set of Fortran codes for parameter search on grid at:'https://github.com/vlcekl/sopt/tree/master/src/optimize/search_grid'
* A Python script for simplex search at https://github.com/vlcekl/sopt/tree/master/src/optimize/search_simplex